In [1]:
import re
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import json
import csv
from datetime import datetime, timedelta, date
from rauth import OAuth2Service
from lyricsgenius import Genius
from clean_colored_lyrics import *
from synced_lyrics import *
# from merge import *
from fuzzysearch import find_near_matches
import pandas as pd
import math
import os
from pathlib import Path
import copy

In [2]:
ACCESS_TOKEN = "WgL1MoH7WJNIR-EdDwj2zhga4YUYwW4ipFOXRIvqn7hVQh6ZLLBhUA4qZr5SypIu"
genius = Genius(ACCESS_TOKEN)

In [3]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    soup = soup.body
    return soup

In [4]:
discography_raw_html = get_soup("https://colorcodedlyrics.com/2015/10/01/seventeen-sebeuntin-lyrics-index/")

In [5]:
print(discography_raw_html.prettify())

<body class="wp-singular post-template-default single single-post postid-81150 single-format-standard wp-custom-logo wp-embed-responsive wp-theme-bloggrid no-sidebar-single" data-rsssl="1">
 <div class="site" id="page">
  <a class="skip-link screen-reader-text" href="#primary">
   Skip to content
  </a>
  <header class="site-header" id="masthead">
   <div class="container">
    <div class="d-flex justify-content-between align-items-center">
     <div class="">
      <div style="align=center">
       <form action="https://colorcodedlyrics.com/" class="search-form" method="get" role="search">
        <label>
         <span class="screen-reader-text">
          Search for:
         </span>
         <input class="search-field" name="s" placeholder="Search …" type="search" value=""/>
        </label>
        <input class="search-submit" type="submit" value="Search"/>
       </form>
       <div class="ko-fi-button" data-code="colorcodedlyrics" data-color="#ED64A6" data-text="Tip Jar" data-ti

In [6]:
korean_eps_html = discography_raw_html.find(id="ub-tabbed-content-19dac54b-4a1f-4737-91c7-d76970079f30-panel-4")
korean_eps_html = korean_eps_html.findAll('div', class_='wp-block-columns is-layout-flex wp-container-core-columns-is-layout-9d6595d7 wp-block-columns-is-layout-flex')

In [7]:
print(korean_eps_html[0].prettify())

<div class="wp-block-columns is-layout-flex wp-container-core-columns-is-layout-9d6595d7 wp-block-columns-is-layout-flex">
 <div class="wp-block-column is-layout-flow wp-block-column-is-layout-flow" style="flex-basis:33.33%">
  <div class="wp-block-image is-resized">
   <figure class="aligncenter size-large">
    <img alt="" class="wp-image-77650" data-recalc-dims="1" decoding="async" height="640" loading="lazy" sizes="auto, (max-width: 640px) 100vw, 640px" src="https://i0.wp.com/colorcodedlyrics.com/wp-content/uploads/2015/05/SEVENTEEN-17-CARAT.jpg?resize=640%2C640&amp;ssl=1" srcset="https://i0.wp.com/colorcodedlyrics.com/wp-content/uploads/2015/05/SEVENTEEN-17-CARAT.jpg?resize=785%2C785&amp;ssl=1 785w, https://i0.wp.com/colorcodedlyrics.com/wp-content/uploads/2015/05/SEVENTEEN-17-CARAT.jpg?resize=250%2C250&amp;ssl=1 250w, https://i0.wp.com/colorcodedlyrics.com/wp-content/uploads/2015/05/SEVENTEEN-17-CARAT.jpg?resize=300%2C300&amp;ssl=1 300w, https://i0.wp.com/colorcodedlyrics.com/wp-

In [8]:
discography = []

for p in korean_eps_html:
    divs = p.findAll('div', recursive=False)
    p = divs[1]
    
    album_name = p.find('span').text

    if album_name.find(']'):
        album_name = album_name[album_name.find(']')+1:]

    tracks = []
    for t in p.findAll('a'):
        track_name = t.text
        if track_name == 'OT13 Ver.':
            continue
        
        track_name = track_name.replace('’',"'")
        track_url = t.get('href')
        tracks.append({'track_name': track_name, 'track_url': track_url})
    discography.append({'album_name': album_name.strip(), '_type': 'Mini Album', 'tracks': tracks})



In [9]:
for d in discography:
    print(d)
    print()

{'album_name': '17 CARAT', '_type': 'Mini Album', 'tracks': [{'track_name': 'Shining Diamond', 'track_url': 'https://colorcodedlyrics.com/2015/06/seventeen-sebeuntin-shining-diamond'}, {'track_name': '아낀다 (Adore U)', 'track_url': 'https://colorcodedlyrics.com/2015/05/seventeen-sebeuntin-adore-u-agginda'}, {'track_name': 'Ah Yeah', 'track_url': 'https://colorcodedlyrics.com/2015/06/seventeen-sebeuntin-ah-yeah'}, {'track_name': 'Jam Jam', 'track_url': 'https://colorcodedlyrics.com/2015/06/seventeen-sebeuntin-jam-jam'}, {'track_name': '20', 'track_url': 'https://colorcodedlyrics.com/2015/06/seventeen-sebeuntin-20'}]}

{'album_name': 'BOYS BE', '_type': 'Mini Album', 'tracks': [{'track_name': '표정관리 (Fronting)', 'track_url': 'https://colorcodedlyrics.com/2015/10/seventeen-fronting-pyojeonggwanli'}, {'track_name': '만세 (MANSAE)', 'track_url': 'https://colorcodedlyrics.com/2015/09/seventeen-mansae-manse'}, {'track_name': '어른이 되면 (When I Grow Up)', 'track_url': 'https://colorcodedlyrics.com/201

In [10]:
def get_lyrics(raw_html, debug):
    lyrics_container = raw_html.findAll('div', class_='Lyrics__Container-sc-68a46031-1 ibbPVY')

    found_section = False
    section_style = []
    assigned_lyrics = []
    members = []
    section = ''
    line = 0
    style_start = False

    for section in lyrics_container:
        if debug: print(section)
        for l in section:
            if l.name == 'div':
                continue
            
            if l.name == 'br':
                if debug: print('br')
                line += 1
                continue
            
            l_style = [l.name]
            l_text = l.get_text()
            if l_text.find('*') >= 0:
                l_style = ['*']
                if l_text.find('**') >= 0:
                    l_style = ['**']

            if l_text == '' or l_text.strip() == ',':
                continue
            
            if not l.name == None:
                for x in l.contents:
                    if not x.name == None and (x.name == 'i' or x.name == 'b'):
                        if debug: print("CHILDREN", x, x.name)
                        l_style.append(x.name)
                l_text = l.get_text(separator="\n", strip=True)
            
            if debug: print(l_style, l_text, found_section)

            if not found_section:
                if l_text.find('[') >= 0:
                    section_style = []
                    section = ''
                    line = 0
                    found_section = True

                    if l_text.find(']') >= 0:
                        found_section = False
                        l_text = l_text[:l_text.find(']')]
                    
                    section = l_text[1:l_text.find(':')]

                    member = l_text[l_text.find(':')+1:].replace(',','').strip()
                
                    if '8' in member:
                        i = member.find('8')
                        if member[i-1] == ' ':
                            member = member[:i-1] + member[i:]
                    
                    member = re.split(r'[&,\s]+', member)
                    if debug: print('new me', {'style': l_style, 'member': member})
                    section_style.append({'style': l_style, 'member': member})

                    for m in member:
                        if m not in members:
                            members.append(m)

                else:
                    curr_member = next((style['member'] for style in section_style if set(style['style']) == set(l_style)), None)
                    
                    if l_style[0] == '*' or l_style[0] == '**':
                        l_text = l_text.replace(l_style[0], '')
                    l_text = l_text.replace('(', '').replace(')', '').strip()
                    
                    while not l_text == '':
                        i = l_text.find('\n')
                        if i == -1:
                            if debug: print('ADD LINE ---> ',{'member': curr_member, 'lyric': l_text, 'section': section, 'line': line})
                            assigned_lyrics.append({'member': curr_member, 'lyric': l_text, 'section': section, 'line': line})
                            # line += 1
                            break
                        curr_t = l_text[:i]

                        l_text = l_text[i+1:]
                        if debug: print('ADD LINE ---> ',{'member': curr_member, 'lyric': curr_t, 'section': section, 'line': line})
                        assigned_lyrics.append({'member': curr_member, 'lyric': curr_t, 'section': section, 'line': line})
                        line += 1
            else:
                if l_text == ']':
                    found_section = False
                    continue

                if l_text.find(']') >= 0:
                    found_section = False
                    l_text = l_text[:l_text.find(']')]

                if debug: print('found', found_section, l_text)
                
                if l_text.find('*') >= 0:
                    pattern = r'(\*\*|\*)([^*]+)\1'
                    for marker, name in re.findall(pattern, l_text):
                        l_style = "**" if marker == "**" else "*"
                        pattern = fr'{re.escape(l_style)}(.*?){re.escape(l_style)}'
                        member = re.findall(pattern, l_text)[0]

                        if '8' in member:
                            i = member.find('8')
                            if member[i-1] == ' ':
                                member = member[:i-1] + member[i:]
                        member = re.split(r'[&,\s]+', member)
                        section_style.append({'style': [l_style], 'member': member})
                        if debug: print('new me', {'style': [l_style], 'member': member}, name)

                        for m in member:
                            if m not in members:
                                members.append(m)
                else:
                    member = l_text[l_text.find(':')+1:].replace(',','').strip()

                    if '8' in member:
                        i = member.find('8')
                        if member[i-1] == ' ':
                            member = member[:i-1] + member[i:]
                    member = re.split(r'[&,\s]+', member)
                    if debug: print('new me', {'style': l_style, 'member': member})
                    section_style.append({'style': l_style, 'member': member})

                    for m in member:
                        if m not in members:
                            members.append(m)

            if debug: print()

        if debug: print()

    return {'assigned_lyrics': assigned_lyrics, "members": members}

# results = get_lyrics(raw_html, True)

In [11]:
def merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug):    
    c_i = 0
    s_i = 0
    m_i = 0

    # colored_lyrics_copy = copy.deepcopy(colored_lyrics[:20])
    # synced_lyrics_copy = copy.deepcopy(synced_lyrics[:20])
    
    colored_lyrics_copy = copy.deepcopy(colored_lyrics)
    synced_lyrics_copy = copy.deepcopy(synced_lyrics)

    merged_lines = copy.deepcopy(colored_lyrics_copy)
    for m in merged_lines:
        m['lyric'] = m['lyric'].strip()
        m['start'] = ''
        m['end'] = ''
        m['delta'] = 0

    unmerged_line_color = []
    unmerged_line_synced = []

    c_looking_ahead_index = 0
    s_looking_ahead_index = 0

    c_len_shorter = True
    check_color = True

    while c_i < len(merged_lines) or s_i < len(synced_lyrics_copy):
        if debug: print('c_i',c_i, '  s_i', s_i, '  m_i', m_i, '  c_looking_ahead_index', c_looking_ahead_index, '    s_looking_ahead_index', s_looking_ahead_index)

        if c_i >= len(merged_lines) or c_looking_ahead_index > looking_ahead_max:
            if debug: print('MERGE REMAINING SYNCED')
            for s in synced_lyrics_copy[s_i:]:
                unmerged_line_synced.append(s)
                if debug: print('ADDING UMERGED SYNCED --->', s)
            break

        if s_i >= len(synced_lyrics_copy) or s_looking_ahead_index > looking_ahead_max:
            if debug: print('MERGE REMAINING COLOR')
            for c in merged_lines[c_i:]:
                unmerged_line_color.append(c)
                if debug: print('ADDING UMERGED COLOR --->', c)
            break


        if c_i + c_looking_ahead_index >= len(merged_lines):
            if debug: print("OVER COLOR")
            looking_ahead_max = max(0, len(merged_lines) - c_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            break
        elif s_i + s_looking_ahead_index >= len(synced_lyrics_copy):
            if debug: print("OVER SYNCED")
            looking_ahead_max = max(0, len(synced_lyrics_copy) - s_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            break

        c_line = colored_lyrics_copy[c_i + c_looking_ahead_index]
        s_line = synced_lyrics_copy[s_i + s_looking_ahead_index]
        c_lyric = c_line['lyric']
        c_lyric_norm = normalize_lyrics(c_lyric)
        s_lyric = s_line['lyric']
        s_lyric_norm = normalize_lyrics(s_lyric)
        m_line = merged_lines[m_i]
        if debug: print(c_line, s_line, m_line)
        if debug: print(c_lyric_norm, s_lyric_norm)

        max_l_dist = 5
        c_len = len(c_lyric_norm)
        s_len = len(s_lyric_norm)
        if c_len < 5 or s_len < 5:
            max_l_dist = 1
        elif c_len < 10 or s_len < 10:
            max_l_dist = int(min(c_len, s_len) / 3)

        find_match = []
        remaining_match = []
        remaining_str = ''
        if c_len <= s_len:
            c_len_shorter = True
            find_match = find_near_matches(c_lyric_norm, s_lyric_norm, max_l_dist=max_l_dist)
            remaining_str = s_lyric_norm
            if debug: print('c_len find_match: ', find_match)
        else:
            c_len_shorter = False
            find_match = find_near_matches(s_lyric_norm, c_lyric_norm, max_l_dist=max_l_dist)
            remaining_str = c_lyric_norm
            if debug: print('s_len find_match: ', find_match)

        if debug: print('found', find_match)
        found = False
        match_end = 0

        if find_match:
            found = find_match[0].start <= 4 and find_match[0].matched
            match_end = find_match[0].end
        else:
            prefix = os.path.commonprefix([c_lyric, s_lyric])
            if debug: print('PREFIX: ' ,prefix)
            if len(prefix) >= 4: 
                if debug: print('STARTS THE SAME')
                match_end = len(prefix)
                found = True
        remaining_str = remaining_str[match_end:].strip()

        if found:
            
            for i in range(s_looking_ahead_index):
                new_line = {
                    'member': ['adlib'],
                    'lyric': synced_lyrics_copy[s_i+i]['lyric'],
                    'start': synced_lyrics_copy[s_i+i]['start'],
                    'end': synced_lyrics_copy[s_i+i]['end'],
                    'delta': synced_lyrics_copy[s_i+i]['delta'],
                    'section': colored_lyrics_copy[max(0,c_i + c_looking_ahead_index - 1)]['section'],
                    'line': colored_lyrics_copy[max(0,c_i + c_looking_ahead_index - 1)]['line'],
                    # 'section_name': ''
                }
                merged_lines.insert(m_i + i, new_line)
                if debug: print("INSERT SYNCED MERGE --->", m_i + i, new_line )
                m_i += 1

            if c_looking_ahead_index and not m_line['lyric'] == c_line['lyric']:
                if debug: print('SKIPPING COLOR -->', m_i, m_line)
                m_i += c_looking_ahead_index

            m_line = merged_lines[m_i]
            if debug: print('NEW M:', m_i , m_line)

            if c_len_shorter and not remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                m_line['end'] = s_line['end']
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)
                if debug: print('EXACT COLOR MERGE --->', m_i, m_line)
                c_i += (1 + c_looking_ahead_index)
                s_i += (1 + s_looking_ahead_index)
                m_i += 1
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0

            if not c_len_shorter and not remaining_str:
                

                m_line = merged_lines[m_i]
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                m_line['end'] = s_line['end']
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)

                if debug: print('EXACT SYNCED MERGE --->', m_i, m_line)
                c_i += 1
                s_i += (1 + s_looking_ahead_index)
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0
                m_i += 1

            if c_len_shorter and remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']

                ratio = float(match_end) / float(len(s_lyric_norm))
                d = round(s_line['delta'] * ratio, 2)
                if debug: print(s_line['start'])
                time_object = datetime.strptime(s_line['start'], '%M:%S:%f').time()
                new_start = str(((datetime.combine(datetime.now(), time_object)) + timedelta(seconds=d)).time())
                if debug: print('new_start', s_line['start'], (datetime.combine(datetime.now(), time_object)), new_start, d)

                if new_start.find('.') > 0:
                    pattern = r"\d{2}:(\d{2}:\d{2}:\d{2})"
                    new_start = re.findall(pattern, new_start.replace('.', ':'))[0]
                else:
                    pattern = r"(\d{2}:\d{2}:\d{2})"
                    new_start = re.findall(pattern, new_start)[0]
                if debug: print('new_start', s_line['start'], (datetime.combine(datetime.now(), time_object)), new_start, d)
                s_line['start'] = new_start
                s_line['delta'] = round(s_line['delta'] - d, 2)
                s_line['lyric'] = remaining_str

                m_line['delta'] = round(m_line['delta'] + d, 2)
                m_line['end'] = new_start
                check_color = True
                                
                c_i += 1
                s_i += s_looking_ahead_index
                s_looking_ahead_index = 0

                if debug: print('PARTIAL S_LINE MERGE --->', m_i, m_line)
                m_i += 1

            if not c_len_shorter and remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                c_line['lyric'] = remaining_str
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)

                time_object = datetime.strptime(s_line['start'], '%M:%S:%f').time()
                new_end = str(((datetime.combine(datetime.now(), time_object)) + timedelta(seconds=m_line['delta'])).time())
                if new_end.find('.') > 0:
                    pattern = r"\d{2}:(\d{2}:\d{2}:\d{2})"
                    new_end = re.findall(pattern, new_end.replace('.', ':'))[0]
                else:
                    pattern = r"(\d{2}:\d{2}:\d{2})"
                    new_end = re.findall(pattern, new_end)[0]

                m_line['end'] = new_end

                s_i += 1
                c_i += c_looking_ahead_index
                c_looking_ahead_index = 0
                check_color = True

                if debug: print('PARTIAL C_LINE MERGE --->', m_i, m_line)
                # m_i += 1

        else:
            if check_color:
                c_looking_ahead_index += 1
                s_looking_ahead_index = 0
                if debug: print("CHECK COLOR")
            else: 
                s_looking_ahead_index += 1
                c_looking_ahead_index = 0
                if debug: print("CHECK SYNCED")

        if s_looking_ahead_index > looking_ahead_max:
            s_looking_ahead_index = 0
            if not check_color:
                s_i += 1
                # c_i += 1

            new_line = {
                'member': '',
                'lyric': s_line['lyric'],
                'start': s_line['start'],
                'end': s_line['end'],
                'delta': s_line['delta'],
                'section': '',
                'line': '',
                # 'section_name': ''
            }
            merged_lines.insert(m_i, new_line)
            check_color = True
            if debug: print("reset all INSERT MERGE --->", m_i, new_line )
            m_i += 1

        if c_looking_ahead_index > looking_ahead_max:
            c_looking_ahead_index = 0
            if not check_color:
                c_i += 1
            else:
                check_color = False
            if debug: print("c_looking_ahead_index reset")
        
        if s_looking_ahead_index > looking_ahead_max:
            s_looking_ahead_index = 0
            s_i += 1
            if debug: print("s_looking_ahead_index reset")

        if debug: print()

    key_order = ['start', 'end', 'delta', 'section_name', 'section', 'line', 'member', 'lyric'] 
    merged_lines = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in merged_lines
    ]

    return merged_lines

In [14]:
for d in discography[6:]:
    genius_album = genius.search_album(d['album_name'], "seventeen")
    genius_album = genius_album.to_dict()
    
    d['cover_art_url'] =  genius_album['cover_art_url']
    d['release_date_components'] = genius_album['release_date_components']
    d['release_date_for_display'] = genius_album['release_date_for_display']
    
    for track in d['tracks'][3:]:
        name = track['track_name']
        print(name)
        i = name.find('(Prod.')
        # print('i', i)
        if i >= 0:
            name = name[:i+1]
        i = name.lower().find('ft.')
        # print('i', i)
        if i >= 0:
            name = name[:i-1]
        name.strip()

        print(name)
        genius_song = genius.search_song(name, 'seventeen')
        if genius_song is None:
            print('genius did not find')
            print()
            continue

        song = genius_song.to_dict()

        print(track['track_url'])

        raw_color_coded = get_raw_color_coded_html(track['track_url'])
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        
        english_name = re.sub(r'[^a-zA-Z0-9\s.,!?\'"*-]', '', name).strip()
        print("english_name" , english_name)
        raw_lrclib = get_song_info(english_name, 'seventeen', d['album_name'])

        if raw_lrclib == 'song not found':
            continue

        synced_lyrics = raw_lrclib['syncedLyrics']

        genius_lyrics = genius_song.to_text()
        pattern = pattern = r"\[([^\]:]+:[^\]]+)\]\s*\n(.*?)(?=\n\s*\[|\Z)"
        results = re.findall(pattern, genius_lyrics, re.DOTALL)

        r_i = 0
        r_section = results[r_i][0]
        r_section = r_section[:r_section.find(':')]
        r_line = results[r_i][1][results[r_i][1].find('\n')+1:]
        r_line_start = r_line[:r_line.find('\n')]
        next_line = results[r_i+1][1][:r_line.find('\n')]

        for l in colored_lyrics:
            find_match = find_near_matches(normalize_lyrics(l['lyric']), normalize_lyrics(next_line), max_l_dist=1)
            if find_match and r_line == '':
                if find_match[0].start <= 3:
                    if r_i+1 < len(results):
                        r_i += 1
                        r_section = results[r_i][0][:results[r_i][0].find(":")]
                        r_line = results[r_i][1][results[r_i][1].find('\n')+1:]
                        r_line_start = results[r_i][1][:results[r_i][1].find("\n")]
                        if r_i+1 < len(results):
                            next_line = results[r_i+1][1][:results[r_i+1][1].find("\n")]
            l['section_name'] = r_section
            r_line = r_line[r_line.find('\n')+1:]
            if r_line.find('\n') == -1:
                r_line = ''
        print('done assigning sections')

        merged_lyrics = merge_singular(colored_lyrics, synced_lyrics, 1, True)

        track['english_name'] = english_name
        track['unit'] = raw_color_coded['unit']
        track['duration'] = raw_lrclib['duration']
        track['media'] = song['media']
        track['color_coded_lyrics'] = colored_lyrics
        track['synced_lyrics'] = synced_lyrics
        track['merged_lyrics'] = merged_lyrics
        track['synced_lyrics_source'] = raw_lrclib['source']
        track['media'] = song['media']
        track['custom_performances'] = song['custom_performances']
        print('done')
        print()


    with open('../json/albums/'+d['album_name'].replace(' ','_')+'_meta_data.json', 'w') as file:
        json.dump(d, file, indent=4)
    print('dump to '+'../json/'+d['album_name'].replace(' ','_')+'_meta_data.json')
    print()


Searching for "Heng:garæ" by seventeen...
My My
My My
Searching for "My My" by seventeen...
Done.
https://colorcodedlyrics.com/2020/06/seventeen-my-my
english_name My My
My My seventeen Heng:garæ https://lrclib.net/api/search?artist_name=seventeen&track_name=My_My&albumName=Heng:garæ
file not found ../official_times/My My.srt
done assigning sections
c_i 0   s_i 0   m_i 0   c_looking_ahead_index 0     s_looking_ahead_index 0
{'member': ['Joshua'], 'lyric': 'Yeh 발맞춰 어디론가 Run away', 'section': 0, 'line': 0, 'section_name': 'Verse 1'} {'start': '00:19:19', 'end': '00:21:78', 'lyric': 'Yeh 발맞춰 어디론가 run away', 'delta': 2.59} {'member': ['Joshua'], 'lyric': 'Yeh 발맞춰 어디론가 Run away', 'section': 0, 'line': 0, 'section_name': 'Verse 1', 'start': '', 'end': '', 'delta': 0}
yeh발맞춰어디론가runaway yeh발맞춰어디론가runaway
c_len find_match:  [Match(start=0, end=17, dist=0, matched='yeh발맞춰어디론가runaway')]
found [Match(start=0, end=17, dist=0, matched='yeh발맞춰어디론가runaway')]
NEW M: 0 {'member': ['Joshua'], 'lyric': 'Y

IndexError: list index out of range

In [ ]:
print(discography[:1][0]['tracks'][0]['color_coded_lyrics'])

[{'member': ['Jun'], 'lyric': 'Slip into the diamond life oh!', 'section': 0, 'line': 0, 'section_name': 'Intro'}, {'member': ['Jun'], 'lyric': ' Slip into the diamond life oh!', 'section': 0, 'line': 1, 'section_name': 'Intro'}, {'member': ['Jun'], 'lyric': ' Slip into the diamond life oh!', 'section': 0, 'line': 2, 'section_name': 'Intro'}, {'member': ['Jun'], 'lyric': ' Ah ah ah ah everybody get up', 'section': 0, 'line': 3, 'section_name': 'Intro'}, {'member': ['Mingyu'], 'lyric': 'Slip into the diamond life oh!', 'section': 0, 'line': 4, 'section_name': 'Intro'}, {'member': ['Vernon'], 'lyric': '(Yeah Seventeen rappin the P man)', 'section': 0, 'line': 5, 'section_name': 'Intro'}, {'member': ['Mingyu'], 'lyric': 'Slip into the diamond life oh!', 'section': 0, 'line': 6, 'section_name': 'Intro'}, {'member': ['Vernon'], 'lyric': '(Ha Yeah Let me show you how)', 'section': 0, 'line': 7, 'section_name': 'Intro'}, {'member': ['Mingyu'], 'lyric': 'Slip into the diamond life oh!', 'secti

In [ ]:
with open('../json/discography_meta_data.json', 'w') as file:
    json.dump(discography[:1], file, indent=4)

In [ ]:
genius_song_raw = genius.search_song('shining diamond', 'seventeen')

Searching for "shining diamond" by seventeen...
Done.


In [ ]:
genius_song = genius_song_raw.to_text()
# print(genius_song)
pattern = pattern = r"\[([^\]:]+:[^\]]+)\]\s*\n(.*?)(?=\n\s*\[|\Z)"
results = re.findall(pattern, genius_song, re.DOTALL)
for r in results:
    print(r)

('Intro: Jun, Vernon', 'Slip into the diamond life, oh!\nSlip into the diamond life, oh!\nSlip into the diamond life, oh!\nAh, ah, ah, everybody get up\nSlip into the diamond life, oh!\nYeah, Seventeen, rappin the P, man\nSlip into the diamond life, oh!\nHa! Yeah, let me show you how\nSlip into the diamond life, oh!\nIt’s post to be done right now\nAh, ah, ah, everybody get up')
('Verse 1: Vernon, S.COUPS', 'Yeah, I know I’m only 17\nI only got a few dollars\n하지만 전혀 개의치 않아 it’d be no problem\n이미 충분한 시간과 압력\n모두 거쳤으니 잘 봐 I’m that diamond\n감탄하게 될 우리 단단함이\n만들어내 가는 기막힌 이 조화보면\n너도 좋아할 꺼야 분명\n내가 눈에 띄면 엄지 들고 붙어, uh')
('Pre-Chorus: Hoshi, WOOZI', '자 눈떠봐, ooh-ah\nLike Jackson I dance around (Eh-eh)\n이제 준비된 사람들\n여기 하나 둘 셋 넷 (Everybody get up)\n흙 속에 묻혀 있던 날\n위로 끌어올려 (Eh-eh)\n이제 빛을 낼 시간이야 oh, oh, oh, oh')
('Chorus: DK, SeungKwan', 'Shining Diamonds, yeah\n시간이 더 흘러도\n약해지지 않아 we’ll keep it up\n지금 이 빛나는 약속을\n네 손에 끼워줄게\nShining Diamonds, yeah, 널 빛낼게\nShining Diamonds, yeah')
('Verse 2: Wonwoo, DK, Ming

In [ ]:
color_lyrics = copy.deepcopy(discography[:1][0]['tracks'][0]['color_coded_lyrics'])
r_i = 0
r_section = results[r_i][0]
r_section = r_section[:r_section.find(':')]
r_line = results[r_i][1][results[r_i][1].find('\n')+1:]
r_line_start = r_line[:r_line.find('\n')]
next_line = results[r_i+1][1][:r_line.find('\n')]

for l in color_lyrics:
    find_match = find_near_matches(normalize_lyrics(l['lyric']), normalize_lyrics(next_line), max_l_dist=1)
    if find_match and r_line == '':
        if find_match[0].start <= 3:
            print(find_match)
            if r_i+1 < len(results):
                r_i += 1
                r_section = results[r_i][0][:results[r_i][0].find(":")]
                r_line = results[r_i][1][results[r_i][1].find('\n')+1:]
                r_line_start = results[r_i][1][:results[r_i][1].find("\n")]
                if r_i+1 < len(results):
                    next_line = results[r_i+1][1][:results[r_i+1][1].find("\n")]
    
    print('section:',r_section, '   line:', r_line_start)
    l['section_name'] = r_section
    r_line = r_line[r_line.find('\n')+1:]
    if r_line.find('\n') == -1:
        r_line = ''
    print(l)

    print()


section: Intro    line: Slip into the diamond life, oh!
{'member': ['Jun'], 'lyric': 'Slip into the diamond life oh!', 'section': 0, 'line': 0, 'section_name': 'Intro'}

section: Intro    line: Slip into the diamond life, oh!
{'member': ['Jun'], 'lyric': ' Slip into the diamond life oh!', 'section': 0, 'line': 1, 'section_name': 'Intro'}

section: Intro    line: Slip into the diamond life, oh!
{'member': ['Jun'], 'lyric': ' Slip into the diamond life oh!', 'section': 0, 'line': 2, 'section_name': 'Intro'}

section: Intro    line: Slip into the diamond life, oh!
{'member': ['Jun'], 'lyric': ' Ah ah ah ah everybody get up', 'section': 0, 'line': 3, 'section_name': 'Intro'}

section: Intro    line: Slip into the diamond life, oh!
{'member': ['Mingyu'], 'lyric': 'Slip into the diamond life oh!', 'section': 0, 'line': 4, 'section_name': 'Intro'}

section: Intro    line: Slip into the diamond life, oh!
{'member': ['Vernon'], 'lyric': '(Yeah Seventeen rappin the P man)', 'section': 0, 'line':